<a href="https://colab.research.google.com/github/hamidriasat/Natural-Language-Processing/blob/master/Incident_Project_XLnet_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Incident Project 
Model deployment code file

First get pretrained model files


In [ ]:
!gdown --id 1-IloNcgJwAWbtqY1pnTzpHi0wdZz_8GI
!gdown --id 1--kmf7_p0_hfyaftSbwik8qAM3V-g46t
!gdown --id 1-IF307LrOjeJbIPszIHhNXRYcWmk9Tg5
!gdown --id 1-Nh3tJxPmQlS0yyKyGeHdPCpQcEv05CP
!gdown --id 1-IRLs4UsIxtWE7fofiV5VF8VdMyEu82c
!gdown --id 1-A0iOh4NL7g1sMBEM5KTZNQ_LTPnYYFC

Downloading...
From: https://drive.google.com/uc?id=1-IloNcgJwAWbtqY1pnTzpHi0wdZz_8GI
To: /content/added_tokens.json
100% 2.00/2.00 [00:00<00:00, 1.76kB/s]
Downloading...
From: https://drive.google.com/uc?id=1--kmf7_p0_hfyaftSbwik8qAM3V-g46t
To: /content/config.json
100% 712/712 [00:00<00:00, 627kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-IF307LrOjeJbIPszIHhNXRYcWmk9Tg5
To: /content/pytorch_model.bin
469MB [00:02, 193MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-Nh3tJxPmQlS0yyKyGeHdPCpQcEv05CP
To: /content/special_tokens_map.json
100% 202/202 [00:00<00:00, 792kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-IRLs4UsIxtWE7fofiV5VF8VdMyEu82c
To: /content/spiece.model
100% 798k/798k [00:00<00:00, 107MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-A0iOh4NL7g1sMBEM5KTZNQ_LTPnYYFC
To: /content/tokenizer_config.json
100% 42.0/42.0 [00:00<00:00, 31.3kB/s]


# install pytorch-transformers libarary

and make all the necessary imports

In [ ]:
!pip install pytorch-transformers


     |████████████████████████████████| 184kB 34.2MB/s 
     |████████████████████████████████| 870kB 43.7MB/s 
     |████████████████████████████████| 1.0MB 52.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=502bac1ad9e5986b7f6594b346d008771eaa7483b23551a326fbc2285cb78f91
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [ ]:
import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import numpy as np

from pytorch_transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Using TensorFlow backend.


# Define model parameters

In [ ]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES["xlnet"]

# Load model in testing mode

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
path_to_modle= "/content/"
model = model_class.from_pretrained( path_to_modle)
tokenizer = tokenizer_class.from_pretrained( path_to_modle)
# Copy the model to the GPU.
model.to(device)
model.eval() #in evaluation mode


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

# Get Testing dataset
For reference we are using training data

You can change and load data according to your requirements


In [ ]:
!gdown --id 1v21uOScs8OScfkldm_7KjpN4zj-S-VI6

!unzip OverAllDataset


Downloading...
From: https://drive.google.com/uc?id=1v21uOScs8OScfkldm_7KjpN4zj-S-VI6
To: /content/OverAllDataset.zip
100% 1.15M/1.15M [00:00<00:00, 107MB/s]
Archive:  OverAllDataset.zip
   creating: OverAllDataset/
  inflating: OverAllDataset/1.txt    
  inflating: OverAllDataset/10.txt   
  inflating: OverAllDataset/100.txt  
  inflating: OverAllDataset/1000.txt  
  inflating: OverAllDataset/1001.txt  
  inflating: OverAllDataset/1002.txt  
  inflating: OverAllDataset/1003.txt  
  inflating: OverAllDataset/1004.txt  
  inflating: OverAllDataset/1005.txt  
  inflating: OverAllDataset/1006.txt  
  inflating: OverAllDataset/1007.txt  
  inflating: OverAllDataset/1008.txt  
  inflating: OverAllDataset/1009.txt  
  inflating: OverAllDataset/101.txt  
  inflating: OverAllDataset/1010.txt  
  inflating: OverAllDataset/1011.txt  
  inflating: OverAllDataset/1012.txt  
  inflating: OverAllDataset/1013.txt  
  inflating: OverAllDataset/1014.txt  
  inflating: OverAllDataset/1015.txt  
  inflat

# Data Input
you can give one or multiple data points at the same time for classification

Output is sequential, meaning order of output will be same as order of input data points

In [ ]:
documents= ["OverAllDataset/1.txt", "OverAllDataset/2.txt", "OverAllDataset/3.txt"
, "OverAllDataset/4.txt", "OverAllDataset/5.txt"]

# Load Input Data 
here like training data is loaded from files

ouput of readData function is a list containing the text of input documents

In [ ]:
# Set the maximum sequence length. 
MAX_LEN = 250

def readData( ):
    full_date= []
    for doc_name in documents:
        with open( doc_name , encoding="cp1252") as content_file:
            fileData= content_file.read()
            templength= len(fileData.split() )
            if templength < MAX_LEN:
                full_date.append(fileData )
            else:
                temp= ""
                convert= fileData.split()
                for j in range(0, MAX_LEN):
                    temp= temp+ " "+convert[j]
                full_date.append(temp )

    return full_date


data_X= readData()


# Preprocessing Data

In [ ]:
# Create sentence and label lists
sentences = data_X

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

# Convert all of our data into torch tensors, the required datatype for our model

input_ids = torch.tensor(input_ids).to(device)
attention_masks = torch.tensor(attention_masks).to(device)


In [ ]:
# Set the batch size.  
batch_size = 16  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size
                                   ,shuffle= False)

# Prediction


In [ ]:

# Tracking variables 
predictions = []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  logits= np.argmax(logits, axis=1).flatten()
  
  # Store predictions and true labels
  predictions += list(logits)
print(' DONE.')

 DONE.


# Result
output is category from 0 to 4

**Dataset Labeling:**

**0**	If no death and major/minor injuries occur in accident

**1**	If only minor injuries occur during accident

**2**	If minor plus major injuries occur during accident

**3**	If only 1 death occur in accident

**4**	If more than 1 death with large number of minor plus major injuries occur during accident


In [ ]:
print("Document #", "==>","Predicted Category")

for i in range(len(predictions)):
  print("Document",(i+1), "==>",predictions[i])


Document # ==> Predicted Category
Document 1 ==> 4
Document 2 ==> 1
Document 3 ==> 1
Document 4 ==> 1
Document 5 ==> 2
